In [1]:
import tabula
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

In [54]:


def insere_data(df,data):
    df['Data'] = data
    df['Data'] = pd.to_datetime(df['Data'].astype(str), format='%d-%m-%Y')
    
    return df

def geraSepultamentos(local, data):
    #gerando o df sem 
    sepultamentos = tabula.read_pdf(local, area=(0, 517, 594, 845), pages ='1'
            ,multiple_tables=True,stream=True, pandas_options={'header':None})
    sepultamentos = sepultamentos[0]
    #
    sepultamentos = sepultamentos.rename(columns ={0:'numero'})
    
    sepultamentos = sepultamentos[sepultamentos.numero.isin(['Número TOTAL de sepultamentos por dia', 'Número TOTAL de óbitos domicliliares por dia','Número TOTAL de cremações por dia'])]
    sepultamentos = sepultamentos.dropna(axis=1, how='all')
    sepultamentos.columns = ['Dado','Quantidade']
    sepultamentos = sepultamentos.set_index('Dado').T
    sepultamentos.columns = ['Sepultamentos','Obitos Domiciliares','Cremações']
    sepultamentos = insere_data(sepultamentos,data)
    
    return sepultamentos

def geraCSV(destino,operacao,dfLeitos,csv_completo):
    if(operacao == 'n'):
        nome = str(destino+'SEPULTAMENTOS.csv')
        dfLeitos.to_csv(nome)
        print('Finalizada a criação do csv!')
    else:
        #recupera o df completo a ser concatenado
        dfCompleto = pd.read_csv(csv_completo)
        dfFinal = pd.concat([dfLeitos,dfCompleto], ignore_index = True)
        dfFinal = dfFinal.drop('Unnamed: 0', axis = 1)
        nome = str(csv_completo)
        dfFinal.to_csv(nome)
        print('Finalizada a concatenação do csv!')



In [52]:
operacao = input('(c) Concatenar | (n) Criar novo csv: ')

operacao = operacao.lower()

if(operacao == 'n'):
    origem = input('Digite o caminho para o PDF: ')

    data = input('Digite a data do arquivo (MES-DIA-ANO): ')

    destino = input('Digite o caminho para ser salvo: ')
    
    #chama a função geraSepultamentos

    sepultamentos = geraSepultamentos(origem,data)
    
    #gera o csv
    
    geraCSV(destino,operacao,sepultamentos,0)
    
else:
    origem = input('Digite o caminho para o PDF: ')

    data = input('Digite a data do arquivo (MM-DD-YYYY): ')

    csv_completo = input('Digite o caminho para o CSV a ser concatenado: ')
    
    #cria o novo df
    sepultamentos = geraSepultamentos(origem,data)
    
    #salva o CSV
    geraCSV(0,operacao,sepultamentos,csv_completo)

(c) Concatenar | (n) Criar novo csv: c
Digite o caminho para o PDF: 03_06_20_BOLETIM_DIARIO_DE_CASOS_COVID-19_v.06-1.pdf
Digite a data do arquivo (MM-DD-YYYY): 06-03-2020
Digite o caminho para o CSV a ser concatenado: dadosSEPULTAMENTOS.csv


Got stderr: jun 25, 2020 11:49:34 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFORMAÇÕES: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored



Finalizada a concatenação do csv!


In [53]:
dados = pd.read_csv('dadosSEPULTAMENTOS.csv')
dados

,Unnamed: 0,Sepultamentos,Obitos Domiciliares,Cremações,Data
0,0,40,6,1,2020-06-03 00:00:00
1,1,40,9,0,2020-06-02 00:00:00
2,2,45,9,1,2020-06-01
